In [72]:
import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
import glob
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import PolynomialFeatures
import re
from scipy import stats
from skimage.viewer import ImageViewer
import cv2
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.externals import joblib
import subprocess
import time
import statsmodels.regression.linear_model as sm
from statsmodels.api import OLS



In [2]:
from RecSystemKM import RecSystemKM_test

In [3]:

path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/'
df = pd.read_csv(path+'users_followers_merged.csv')

### pull a sample for the pipeline
df = df[2:4]

# Scrape pics from users - don't rerun

for i, v in enumerate(df.username): 
    time.sleep(20)
    subprocess.check_output(["instaloader", (str(v))])
    

# Create the Engagment pipeline

In [ ]:


temp_path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/User_pics/'
train_path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/User_pics/'
save_path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/User_pics/'
dest_img =  '/Users/kimia/Desktop/Capstone/hotel/Rec_system/User_pics/'

userlist= ['or_shmul','fathy_abo_dahis', 'iamkurdapya', 'abhishekraghav015',  'jnbax']
#user list will be the picture number! 


# Vectorize images 

In [5]:
c = RecSystemKM_test(train_path, save_path )
c.Karans_get_train_images(userlist)


#Create Object

train_df = c.convert_to_features(columns = ["User_Handle","URL","R_Mean", "R_STD", "R_MED", "G_Mean", "G_STD", 
                "G_MED", "B_Mean", "B_STD", "B_MED", "Canny", "ORB_X", "ORB_Y"])

Number of images loaded: 302


In [25]:
# now merge with # of followers

In [56]:
path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/'
df_usernames = pd.read_csv(path+'users_followers_w_bax.csv')


df_usernames['followers'] = (df_usernames['followers'].apply(lambda x: x.replace('k',"000").replace(".","").replace("m","000000").replace(",","")))

df_usernames.followers = pd.to_numeric(df_usernames.followers)

In [57]:
df_usernames = df_usernames[0:6]


In [58]:
df = train_df.merge(df_usernames,how='left',on='User_Handle')

In [59]:
# drop extra columns

In [60]:
remove = ['User_Handle', 'URL', 'Unnamed: 0']
df.drop(remove, axis = 1, inplace = True)

In [61]:
## any null?
print(df.isnull().sum(axis=0))

R_Mean       0
R_STD        0
R_MED        0
G_Mean       0
G_STD        0
G_MED        0
B_Mean       0
B_STD        0
B_MED        0
Canny        0
ORB_X        0
ORB_Y        0
followers    0
dtype: int64


# build a model

In [62]:
rands = np.random.seed(9001)
msk = np.random.rand(len(df)) < 0.25
data_train = df[~msk]
data_test = df[msk]

In [63]:
y_train = data_train.followers
y_test = data_test.followers

predictors = ['R_Mean', 'R_STD', 'R_MED', 'G_Mean', 'G_STD', 'G_MED', 'B_Mean',
       'B_STD', 'B_MED', 'Canny', 'ORB_X', 'ORB_Y',]
X_train = data_train[predictors]
X_test = data_test[predictors]

In [64]:
len(y_train), len(X_train_constant)

(223, 223)

In [80]:


X_train_constant= sm.add_constant(X_train)
X_test_constant= sm.add_constant(X_test)

MODEL = sm.OLS(y_train.astype(float), X_train_constant.astype(float)).fit()
#results = model_train.fit()
from sklearn.metrics import r2_score


In [82]:
yHatTest = MODEL.predict(X_test)
r2_score(y_test, yHatTest)

ValueError: shapes (78,12) and (13,) not aligned: 12 (dim 1) != 13 (dim 0)

In [73]:
alpha= [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2,10**3, ]
ridge = RidgeCV(alphas=alpha, cv=10, fit_intercept=True)
ridge.fit(X_train, y_train)
print(ridge.fit(X_train, y_train))



RidgeCV(alphas=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000], cv=10,
    fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)


In [79]:
ridge.score(X_train, y_train), ridge.score(X_test, y_test)

(0.17703277522611838, -0.07427373566664919)

In [74]:

lasso = LassoCV(alphas=alpha, cv=10, max_iter=10000, fit_intercept=True)
print(lasso.fit(X_train, y_train))


LassoCV(alphas=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    copy_X=True, cv=10, eps=0.001, fit_intercept=True, max_iter=10000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)


In [78]:
lasso.score(X_train, y_train), lasso.score(X_test, y_test)

(0.09840868417371484, 0.01974278516898975)

0.01974278516898975